# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib

In [4]:
##from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl')

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [6]:
wml_credentials = {
                  "url": "https://us-south.ml.cloud.ibm.com",
                "access_key": "qfrIBYCWMCHATnv3fQs2mWPpW5Gr2FWcPOD_35SEsQcN",
                  "username": "b7a767ea-f627-419b-a998-27aebb385374",
                  "password": "c176323d-5c4c-40ad-bb3a-c69cdc583e95",
                  "instance_id": "a3e79f17-4c9a-43cc-9bd7-9b3019355bc5"
                 }

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [7]:
client= WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [8]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Laura", 
               client.repository.ModelMetaNames.NAME: "Modelo de reviews"}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [11]:
Modelo_publicado= client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=None, 
                                                training_target=None)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [12]:
published_model_uid = client.repository.get_model_uid(Modelo_publicado)
model_details = client.repository.get_details(published_model_uid)

In [13]:
models_details = client.repository.list_models()

------------------------------------  -----------------  ------------------------  -----------------
GUID                                  NAME               CREATED                   FRAMEWORK
61a29e94-e1ef-4f14-8658-c476ecad8cfc  Modelo de reviews  2019-07-29T15:50:34.359Z  scikit-learn-0.20
------------------------------------  -----------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [14]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [15]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.8550000000000002
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       200
           1       0.83      0.89      0.86       200

   micro avg       0.85      0.85      0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400

